In [10]:
%%capture
%pip install -r ../requirements.txt


In [11]:
from schema.WLD import WLD

wld = WLD(file_path="../data/example-chappy.wld")


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict

# Extract and sort workout dates
workout_dates = sorted([w.date for w in wld.workouts])

print(f"Total workouts: {len(workout_dates)}")
print(f"Date range: {workout_dates[0].strftime('%Y-%m-%d')} to {workout_dates[-1].strftime('%Y-%m-%d')}")
print(f"Total days span: {(workout_dates[-1] - workout_dates[0]).days} days")


Total workouts: 2082
Date range: 2017-08-10 to 2025-10-28
Total days span: 3000 days


In [13]:
def calculate_streaks(workout_dates, max_gap_days=1):
    """
    Calculate workout streaks based on calendar days.
    A streak continues if there are <= max_gap_days between workouts.
    """
    if not workout_dates:
        return [], []

    streaks = []
    gaps = []

    streak_start = workout_dates[0]

    for i in range(1, len(workout_dates)):
        # Convert to date objects for proper calendar day calculation
        date1 = workout_dates[i-1].date() if hasattr(workout_dates[i-1], 'date') else workout_dates[i-1]
        date2 = workout_dates[i].date() if hasattr(workout_dates[i], 'date') else workout_dates[i]
        days_diff = (date2 - date1).days

        if days_diff > max_gap_days:
            # End current streak
            streak_end = workout_dates[i-1]
            # Convert to date objects for calendar day calculation
            start_date = streak_start.date() if hasattr(streak_start, 'date') else streak_start
            end_date = streak_end.date() if hasattr(streak_end, 'date') else streak_end
            streak_days = (end_date - start_date).days + 1
            workout_count = sum(1 for d in workout_dates if streak_start <= d <= streak_end)
            streaks.append({
                'days': streak_days,
                'workouts': workout_count,
                'start': streak_start,
                'end': streak_end
            })

            # Record gap
            gaps.append({
                'days': days_diff,
                'start': workout_dates[i-1],
                'end': workout_dates[i]
            })

            # Start new streak
            streak_start = workout_dates[i]

    # Add final streak
    streak_end = workout_dates[-1]
    # Convert to date objects for calendar day calculation
    start_date = streak_start.date() if hasattr(streak_start, 'date') else streak_start
    end_date = streak_end.date() if hasattr(streak_end, 'date') else streak_end
    streak_days = (end_date - start_date).days + 1
    workout_count = sum(1 for d in workout_dates if streak_start <= d <= streak_end)
    streaks.append({
        'days': streak_days,
        'workouts': workout_count,
        'start': streak_start,
        'end': streak_end
    })

    return streaks, gaps

streaks, gaps = calculate_streaks(workout_dates)

print(f"\nTotal streaks: {len(streaks)}")
print(f"Total gaps (1+ day): {len(gaps)}")



Total streaks: 515
Total gaps (1+ day): 514


In [14]:
# Check current streak (1-day threshold)
last_workout = workout_dates[-1]
days_since_last = (datetime.now() - last_workout).days

if days_since_last <= 1:
    current_streak = streaks[-1]
    current_streak_days = (datetime.now() - current_streak['start']).days
    print(f"✅ ACTIVE STREAK!")
    print(f"   Current streak: {current_streak_days} days")
    print(f"   Workouts in streak: {current_streak['workouts']}")
    print(f"   Streak started: {current_streak['start'].strftime('%Y-%m-%d')}")
    print(f"   Last workout: {last_workout.strftime('%Y-%m-%d')} ({days_since_last} days ago)")
else:
    print(f"❌ NO ACTIVE STREAK")
    print(f"   Last workout: {last_workout.strftime('%Y-%m-%d')} ({days_since_last} days ago)")
    print(f"   Last streak ended: {streaks[-1]['end'].strftime('%Y-%m-%d')}")
    print(f"   Last streak length: {streaks[-1]['days']} days")


✅ ACTIVE STREAK!
   Current streak: 19 days
   Workouts in streak: 20
   Streak started: 2025-10-09
   Last workout: 2025-10-28 (0 days ago)


In [ ]:
sorted_streaks = sorted(streaks, key=lambda x: x['days'], reverse=True)

print("Top 10 Longest Streaks:")
print("=" * 120)
for i, streak in enumerate(sorted_streaks[:10], 1):
    # Get workouts in this streak
    streak_workouts = [w for w in wld.workouts if streak['start'] <= w.date <= streak['end']]

    # Calculate metrics
    avg_sets = sum(w.numSets() for w in streak_workouts) / len(streak_workouts) if streak_workouts else 0
    avg_volume_k = (sum(w.volume() for w in streak_workouts) / len(streak_workouts) / 1000) if streak_workouts else 0
    avg_duration = sum(w.duration for w in streak_workouts) / len(streak_workouts) if streak_workouts else 0

    # Format duration (duration is in seconds)
    hours = int(avg_duration // 3600)
    minutes = int((avg_duration % 3600) // 60)
    duration_str = f"{hours}h{minutes}m" if hours > 0 else f"{minutes}m"

    print(f"{i:2d}. {streak['days']:3d} days | "
          f"avg {avg_sets:.1f} sets | "
          f"avg {avg_volume_k:.1f}k lbs | "
          f"{duration_str} avg | "
          f"{streak['start'].strftime('%Y-%m-%d')} → {streak['end'].strftime('%Y-%m-%d')}")


Top 10 Longest Streaks:
 1.  50 days | avg 24.4 sets | avg 32.9k lbs | 53m avg | 2023-07-06 → 2023-08-24
 2.  45 days | avg 22.8 sets | avg 27.0k lbs | 55m avg | 2022-08-08 → 2022-09-21
 3.  34 days | avg 16.8 sets | avg 16.4k lbs | 53m avg | 2020-04-13 → 2020-05-16
 4.  30 days | avg 21.0 sets | avg 18.4k lbs | 59m avg | 2017-09-19 → 2017-10-18
 5.  30 days | avg 19.9 sets | avg 14.9k lbs | 51m avg | 2018-01-23 → 2018-02-21
 6.  23 days | avg 23.3 sets | avg 43.0k lbs | 56m avg | 2023-04-19 → 2023-05-11
 7.  20 days | avg 24.9 sets | avg 31.0k lbs | 56m avg | 2025-10-09 → 2025-10-28
 8.  19 days | avg 19.3 sets | avg 21.0k lbs | 46m avg | 2022-07-11 → 2022-07-29
 9.  19 days | avg 25.9 sets | avg 46.1k lbs | 1h5m avg | 2023-06-16 → 2023-07-04
10.  17 days | avg 20.3 sets | avg 22.5k lbs | 52m avg | 2022-04-25 → 2022-05-11
